In [ ]:
import os
from glob import glob

input_dir = "./tree_data/0_RGB_FullyLabeled/coco/val/labels"
output_dir = "./tree_data/0_RGB_FullyLabeled/coco/val/labels" 
os.makedirs(output_dir, exist_ok=True)

label_remap = {
    "1": "3",  # Picea abies
    "2": "6",  # Pinus sylvestris
    "3": "5"   # Betula sp.
}

label_files = glob(os.path.join(input_dir, "*.txt"))
for file_path in label_files:
    new_lines = []
    with open(file_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if parts and parts[0] in label_remap:
                parts[0] = label_remap[parts[0]]
            new_lines.append(" ".join(parts))

    output_path = os.path.join(output_dir, os.path.basename(file_path))
    with open(output_path, "w") as f:
        f.write("\n".join(new_lines))

print(f"✅ Done! {len(label_files)} files processed and saved to:\n{output_dir}")


In [ ]:
import os
from glob import glob

base_root = "./tree_data/"

subdirs = [
    "0_RGB_FullyLabeled",
    "5_RGB_S_320_pL",
    "12_RGB_ObjDet_640_fL",
    "34_RGB_ObjDet_640_pL",
    "34_RGB_ObjDet_640_pL_b"
]

# train, val
for subdir in subdirs:
    for split in ["train", "val"]:
        label_dir = os.path.join(base_root, subdir, split, "labels")
        if not os.path.exists(label_dir):
            continue

        label_files = glob(os.path.join(label_dir, "*.txt"))
        for file_path in label_files:
            new_lines = []
            with open(file_path, "r") as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        try:
                            parts[0] = str(int(parts[0]) - 1)  # class_id - 1
                            new_lines.append(" ".join(parts))
                        except ValueError:
                            continue

            with open(file_path, "w") as f:
                f.write("\n".join(new_lines))

print("class_id -= 1")


In [ ]:
import os
from PIL import Image


SOURCE_ROOT = "./tree_data/0_RGB_FullyLabeled"
TARGET_ROOT = "./tree_data/0_RGB_FullyLabeled_tiles"
SETS = ["train", "val"]
THRESHOLD = 800

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def process_directory(image_dir, label_dir, out_image_dir, out_label_dir):
    ensure_dir(out_image_dir)
    ensure_dir(out_label_dir)

    for filename in os.listdir(image_dir):
        if not filename.lower().endswith(".png"):
            continue

        image_path = os.path.join(image_dir, filename)
        label_name = os.path.splitext(filename)[0] + ".txt"
        label_path = os.path.join(label_dir, label_name)

        try:
            with Image.open(image_path) as img:
                width, height = img.size
                if width <= THRESHOLD and height <= THRESHOLD:
                    continue  # skip small images

                tw, th = width // 2, height // 2

                for i in range(2):
                    for j in range(2):
                        left = j * tw
                        upper = i * th
                        right = left + tw
                        lower = upper + th

                        tile = img.crop((left, upper, right, lower))
                        tile_name = f"{os.path.splitext(filename)[0]}_tile_{i}_{j}.png"
                        tile_path = os.path.join(out_image_dir, tile_name)
                        tile.save(tile_path)

                # process label if exists
                if os.path.exists(label_path):
                    with open(label_path, 'r') as lf:
                        lines = lf.readlines()

                    objects = []
                    for line in lines:
                        parts = line.strip().split()
                        if len(parts) != 5:
                            continue
                        cls, cx, cy, w, h = int(parts[0]), float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4])
                        abs_cx, abs_cy = cx * width, cy * height
                        abs_w, abs_h = w * width, h * height
                        objects.append((cls, abs_cx, abs_cy, abs_w, abs_h))

                    tile_labels = {f"{i}_{j}": [] for i in range(2) for j in range(2)}
                    for cls, cx, cy, w, h in objects:
                        x0, y0 = cx - w/2, cy - h/2
                        x1, y1 = cx + w/2, cy + h/2
                        for i in range(2):
                            for j in range(2):
                                tx, ty = j * tw, i * th
                                tx1, ty1 = tx + tw, ty + th
                                ix0 = max(x0, tx)
                                iy0 = max(y0, ty)
                                ix1 = min(x1, tx1)
                                iy1 = min(y1, ty1)
                                if ix1 > ix0 and iy1 > iy0:
                                    ncx = ((ix0 + ix1) / 2) - tx
                                    ncy = ((iy0 + iy1) / 2) - ty
                                    nw = ix1 - ix0
                                    nh = iy1 - iy0
                                    norm_cx = ncx / tw
                                    norm_cy = ncy / th
                                    norm_w = nw / tw
                                    norm_h = nh / th
                                    tile_labels[f"{i}_{j}"].append((cls, norm_cx, norm_cy, norm_w, norm_h))

                    for key, bboxes in tile_labels.items():
                        tile_label_name = f"{os.path.splitext(filename)[0]}_tile_{key}.txt"
                        tile_label_path = os.path.join(out_label_dir, tile_label_name)
                        with open(tile_label_path, "w") as f:
                            for cls, cx, cy, w, h in bboxes:
                                f.write(f"{cls} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")

                print(f"[✓] Tiled: {filename}")

        except Exception as e:
            print(f"[!] Error processing {filename}: {e}")


for subset in SETS:
    image_dir = os.path.join(SOURCE_ROOT, subset, "images")
    label_dir = os.path.join(SOURCE_ROOT, subset, "labels")
    out_image_dir = os.path.join(TARGET_ROOT, subset, "images")
    out_label_dir = os.path.join(TARGET_ROOT, subset, "labels")

    process_directory(image_dir, label_dir, out_image_dir, out_label_dir)